### Import thư viện


In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

### Load & Overview dữ liệu


#### Load dữ liệu


In [96]:
df = pd.read_csv('../Data/laptop.csv')
df.head(5)

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
0,0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,22990,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4 RAM,2.0 Ghz Processor,LED,11.6,Integrated Graphics,MediaTek,64 GB SSD Storage,No HDD,45,Upto 12 Hrs Battery Life
1,1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,36289,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,65,Upto 11 Hrs Battery Life
2,3,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,78500,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5 RAM,3.3 Ghz Processor,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,56,Upto 10 Hrs Battery Life
3,4,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,55490,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,Iris Xe,Intel,512 GB SSD Storage,No HDD,no,Upto 7.30 Hrs Battery Life
4,6,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,21990,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.7 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,45,Upto 8 Hrs Battery Life


Xem thông tin tóm tắt của dữ liệu


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3976 entries, 0 to 3975
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       3976 non-null   int64 
 1   Brand            3976 non-null   object
 2   Name             3976 non-null   object
 3   Price            3976 non-null   int64 
 4   Processor_Name   3976 non-null   object
 5   Processor_Brand  3976 non-null   object
 6   RAM_Expandable   3976 non-null   object
 7   RAM              3976 non-null   object
 8   RAM_TYPE         3976 non-null   object
 9   Ghz              3976 non-null   object
 10  Display_type     3976 non-null   object
 11  Display          3976 non-null   object
 12  GPU              3968 non-null   object
 13  GPU_Brand        3972 non-null   object
 14  SSD              3976 non-null   object
 15  HDD              3976 non-null   object
 16  Adapter          3976 non-null   object
 17  Battery_Life     3558 non-null   

Kết quả ban đầu cho thấy bộ dữ liệu có 18 cột, trong đó có 1 cột "Unnamed: 0" không có ý nghĩa, còn lại hầu hết là các cột có kiểu dữ liệu chuỗi (object). Còn lại cột "Price" có kiểu dữ liệu là int. Vì đề tài đồ án là phân tích giá bán laptop nên trường thông tin "Price" sẽ là biến mục tiêu. Nhóm sẽ thực hiện phân tích sự ảnh hưởng của các yếu tố khác tới trường dữ liệu này.


Ý nghĩa của các trường dữ liệu:

- Brand: Thương hiệu máy tính xách tay (ví dụ: HP, Lenovo, Dell).
- Name: Tên model cụ thể của máy tính xách tay.
- Giá của máy tính xách tay (theo đơn vị tiền tệ).
- Processor_Name: Tên bộ xử lý được sử dụng (ví dụ: Intel Core i5, AMD Ryzen 5).
- Processor-Brand: Thương hiệu bộ xử lý (ví dụ: Intel, AMD, MediaTek).
- RAM_Expandable: Chỉ ra liệu RAM có thể mở rộng hay không và mở rộng đến mức nào (ví dụ: "12 GB Expandable").
- RAM: Dung lượng RAM được lắp đặt trong máy tính xách tay (ví dụ: 8 GB).
- RAM_TYPE: Loại RAM được sử dụng (ví dụ: DDR4, LPDDR4X).
- Ghz: Tốc độ xung nhịp của bộ xử lý (tính bằng GHz).
- Display_type: Loại màn hình được sử dụng (ví dụ: LCD, LED).
- Display: Kích thước màn hình của máy tính xách tay (tính bằng inch).
- GPU: Mẫu GPU (nếu có), hoặc là "Integrated Graphics" đối với card đồ họa tích hợp vào CPU.
- GPU_Brand: Thương hiệu của GPU (ví dụ: NVIDIA, AMD, Intel).
- SSD: Dung lượng lưu trữ cho SSD (ví dụ: "512 GB SSD Storage").
- HDD: Dung lượng lưu trữ của HDD (Ổ đĩa cứng) (nếu có).
- Adapter: Công suất của bộ chuyển đổi nguồn (tính bằng watt).
- Battery_Life: Tuổi thọ pin dự kiến ​​của máy tính xách tay (ví dụ: "Up to 12 Hrs Battery Life").


In [98]:
# Xem số dòng và cột của bộ dữ liệu
print("Bộ dữ liệu có {} dòng và {} cột".format(df.shape[0], df.shape[1]))

Bộ dữ liệu có 3976 dòng và 18 cột


#### Thống kê và xử lý giá trị bị khuyết


In [99]:
# Thống kê tỉ lệ giá trị khuyết trong các trường thông tin
total_null = df.isna().sum()
percent = total_null/df.shape[0] * 100.0
pd.concat([total_null, percent], axis=1, keys=['Count', 'Percentage (%)']).sort_values(by='Percentage (%)', ascending=False)

,Count,Percentage (%)
Battery_Life,418,10.513078
GPU,8,0.201207
GPU_Brand,4,0.100604
Unnamed: 0,0,0.000000
Brand,0,0.000000
Name,0,0.000000
RAM_Expandable,0,0.000000
Price,0,0.000000
Processor_Name,0,0.000000
Processor_Brand,0,0.000000


Bộ dữ liệu ban đầu có 3 trường bị khuyết là GPU, GPU_Brand và Battery_Life. Hiển thị các dòng có giá trị khuyết để kiểm tra.


In [100]:
null_data = df[df.isna().any(axis=1)]
null_data.head(5)

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,Battery_Life
20,23,HP,HP Pavilion 14-dv2153TU (8Y2S3PA) Laptop (14 I...,69999,Intel Core i5 (12th Gen),Intel,16 GB Expandable,16 GB,DDR4 RAM,4.2 Ghz Processor,LCD,14,Iris Xe,Intel,1024 GB SSD Storage,No HDD,no,NaN
30,34,HP,HP 15-fc0026AU (7L030PA) Laptop (15.6 Inch | A...,36990,AMD Quad-Core Ryzen 3,AMD,Not Expandable,8 GB LP,LPDDR5 RAM,2.4 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,no,NaN
37,41,HP,HP 14s-dq3017TU (3Y0H4PA) Laptop (14 Inch | Ce...,27875,Intel Celeron Dual-Core,Intel,Not Expandable,8 GB,DDR4 RAM,2.8 Ghz Processor,LED,14,UHD,Intel,256 GB SSD Storage,No HDD,no,NaN
62,68,Lenovo,Lenovo Legion 5i (82AU00KLIN) Laptop (15.6 Inc...,53535,Intel Core i5 (10th Gen),Intel,Not Expandable,8 GB,DDR4 RAM,2.5 Ghz Processor,LED,15.6,"Geforce GTX 1650 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,no,NaN
122,137,HP,HP 14s-dq5138tu (9D3M7PA) Laptop (14 Inch | Co...,41490,Intel Core i3 (12th Gen),Intel,16 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,14,UHD,Intel,512 GB SSD Storage,No HDD,no,NaN


Ta nhận thấy rằng cột Battery_Life có tỉ lệ bị khuyết đáng chú ý nên sẽ thực hiện xem xét các giá trị của cột để quyết định xem có thể dùng các phương pháp điền khuyết hay không.


In [101]:
df['Battery_Life'].value_counts()

Battery_Life
65W Adapter                    803
45W Adapter                    287
 Upto 6 Hrs Battery Life       199
 Upto 8 Hrs Battery Life       151
 Upto 10 Hrs Battery Life      144
                              ... 
 Upto 12.5 Hrs Battery Life      1
 Upto 3.5 Hrs Battery Life       1
60.7W Adapter                    1
69W Adapter                      1
71W Adapter                      1
Name: count, Length: 191, dtype: int64

Ta thấy rằng giá trị của Battery_Life nên là số giờ, chẳng hạn như "Upto 6 Hrs Battery Life" thay vì là "65W Adapter" (giá trị này có nghĩa là bộ sạc có công suất 65W) trong khi cột Adapter đã thể hiện ý nghĩa này rồi. Điều này có nghĩa là các giá trị như "65W Adapter không có ý nghĩa trong cột Battery_Life, nên sẽ không có ý nghĩa cho việc hỗ trợ điền khuyết, các giá trị này sẽ được xem là null.


In [102]:
# Chuyển các giá trị chứa 'Adapter' thành NaN trong cột Battery_Life
df['Battery_Life'] = df['Battery_Life'].apply(lambda x: np.nan if 'Adapter' in str(x) else x)
# Kiếm tra lại số lượng khuyết trong cột Battery_Life
null_count = df['Battery_Life'].isna().sum()
print(f"Số lượng khuyết hiện tại trong cột Battery_Life: {null_count}")
print(f"Tỉ lệ bị khuyết trong cột Battery_Life: {(null_count/df.shape[0]).round(3)*100.0}%")

Số lượng khuyết hiện tại trong cột Battery_Life: 2476
Tỉ lệ bị khuyết trong cột Battery_Life: 62.3%


Như vậy cột Battery_Life có tỉ lệ bị khuyết đáng kể, không còn ý nghĩa trong việc dự đoán nên sẽ được loại bỏ khỏi dữ liệu


In [103]:
# loại bỏ cột Battery_Life khỏi dataframe
df = df.drop(columns=['Battery_Life'], axis=1)
df.head(5)

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter
0,0,HP,HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (1...,22990,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4 RAM,2.0 Ghz Processor,LED,11.6,Integrated Graphics,MediaTek,64 GB SSD Storage,No HDD,45
1,1,Lenovo,Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15....,36289,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,65
2,3,Dell,Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch...,78500,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5 RAM,3.3 Ghz Processor,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,56
3,4,HP,HP 15s-fy5007TU (91R03PA) Laptop (15.6 Inch | ...,55490,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,Iris Xe,Intel,512 GB SSD Storage,No HDD,no
4,6,Infinix,Infinix Inbook Y2 Plus XL29 Laptop (15.6 Inch ...,21990,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.7 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,45


Các trường thông tin còn (GPU và GPU_Brand) có số lượng bị khuyết không đáng kể nên sẽ tiến hành các ký thuật điền khuyết hoặc chỉ loại bỏ các dòng bị khuyết


In [104]:
# xem các giá trị của cột GPU
df['GPU'].unique()

array(['Integrated Graphics', 'Radeon', 'GeForce RTX 3050 GPU, 4 GB',
       'Iris Xe', 'UHD', 'UHD 620', 'GeForce RTX 2050 GPU, 4 GB',
       'GeForce RTX 4050 GPU, 6 GB', 'Arc', 'GeForce GTX 1650 GPU, 4 GB',
       'UHD 605', 'GeForce RTX 3060 GPU, 6 GB',
       'GeForce GTX 1650 Max Q GPU, 4 GB',
       'GeForce RTX 3050 Ti GPU, 4 GB', 'Integrated',
       'Geforce GTX 1650 GPU, 4 GB', 'Radeon Vega 3', 'UHD Graphics',
       'UHD GPU, 128 MB', 'M1', 'HD 6000', 'UHD 600', 'Radeon Vega 6',
       'M2 Integrated', 'HD 620', 'Geforce MX150 GPU, 2 GB',
       'Radeon Vega 8', 'Geforce GTX 1650 Max-Q GPU, 4 GB',
       'GeForce GTX 1650 Ti GPU, 4 GB', 'Radeon RX Vega 10', 'HD 520',
       'Radeon RX 5500M GPU, 4 GB', 'GeForce GTX 3050 GPU, 4 GB',
       'Radeon 610M', 'Radeon RX6500M GPU, 4 GB',
       'GeForce GTX 1650 MAX Q GPU, 4 GB', 'GeForce RTX 2050', 'HD 500',
       'Radeon Graphics', 'GeForce RTX 3070 Ti GPU, 8 GB',
       'GeForce GTX 1050 GPU, 4 GB', 'Geforce RTX 3050 GPU, 4 GB

In [105]:
null_data = df[df.isna().any(axis=1)]
null_data.head(5)

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter
1008,1073,Samsung,Samsung Galaxy Book 4 NP750XGK-LG3IN Laptop (1...,79999,Intel Core 7 (Series 1),Intel,Not Expandable,16 GB LP,LPDDR4X RAM,1.8 Ghz Processor,LCD,15.6,NaN,Intel,512 GB SSD Storage,No HDD,45
1052,1118,HP,HP 15-fd1096TU (A03CJPA) Laptop (15.6 Inch | C...,64990,Intel Core 5 (Series 1),Intel,Not Expandable,8 GB,DDR4 RAM,4.8 Ghz Processor,LCD,15.6,NaN,Intel,512 GB SSD Storage,No HDD,65
1117,1185,Samsung,Samsung Galaxy Book 4 NP750XGK-KS2IN Laptop (1...,72990,Intel Core 5 (Series 1),Intel,Not Expandable,16 GB LP,LPDDR4X RAM,1.4 Ghz Processor,LCD,15.6,NaN,Intel,512 GB SSD Storage,No HDD,45
2147,2303,Samsung,Samsung Galaxy Book 4 NP750XGK-KG1IN Laptop (1...,70990,Intel Core 5 (Series 1),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.4 Ghz Processor,LCD,15.6,NaN,Intel,512 GB SSD Storage,No HDD,45
2153,2309,Samsung,Samsung Galaxy Book 4 NP750XGK-KG2IN Laptop (1...,75990,Intel Core 5 (Series 1),Intel,Not Expandable,16 GB LP,LPDDR4X RAM,1.4 Ghz Processor,LCD,15.6,NaN,Intel,512 GB SSD Storage,No HDD,45


Đối với các giá trị bị khuyết của trường GPU, tuy không có giá trị nhưng trường GPU_Brand tương ứng, chủ yếu là 'Intel', nên sẽ xem đó là card tích hợp, điền khuyết bằng giá trị 'Integrated Graphics'


In [106]:
# Điền giá trị 'Integrated Graphics' vào cột GPU nếu GPU bị khuyết và GPU_Brand là 'Intel'
df.loc[df['GPU'].isnull() & (df['GPU_Brand'] == 'Intel'), 'GPU'] = 'Integrated Graphics'
# Kiểm tra giá trị khuyết còn lại trong cột GPU
null_count = df['GPU'].isna().sum()
print(f"Số lượng khuyết hiện tại trong cột GPU: {null_count}")

Số lượng khuyết hiện tại trong cột GPU: 0


In [107]:
# Kiểm tra giá trị khuyết còn lại trong cột GPU_Brand
null_data = df[df.isna().any(axis=1)]
null_data.head(5)

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter
3275,3671,Lenovo,Lenovo essential G50-45 (80E300RGIN) Laptop (1...,32500,2.0 Ghz Processor,2.0,16 GB Expandable,8 GB,DDR3 RAM,0,LED,15.6,"R5 M230 GPU, 2 GB",NaN,NO SSD,1024 GB HDD Storage,65
3491,3901,Lenovo,Lenovo essential G50-45 (80E3014FIN) Laptop (1...,25890,AMD Quad-Core A8 APU,AMD,16 GB Expandable,4 GB,DDR3 RAM,2.0 Ghz Processor,LED,15.6,R5,NaN,NO SSD,500 GB HDD Storage,65
3599,4016,Apple,Apple MacBook Pro MR932HN/A Ultrabook (15.4 In...,187990,Intel Core i7 (8th Gen),Intel,32 GB Expandable,16 GB,DDR4 RAM,2.2 Ghz Processor,LED,15.4,"Pro 555X GPU, 4 GB",NaN,256 GB SSD Storage,No HDD,no
3699,4121,Apple,Apple MacBook Pro MR962HN/A Ultrabook (15.4 In...,199900,Intel Core i7 (8th Gen),Intel,32 GB Expandable,16 GB,DDR4 RAM,2.2 Ghz Processor,LED,15.4,"Pro 555X GPU, 4 GB",NaN,256 GB SSD Storage,No HDD,no


In [108]:
# Xem các brand hiện có của GPU
df['GPU_Brand'].value_counts()

GPU_Brand
Intel        1939
NVIDIA       1380
AMD           616
Apple          21
Nvidia          8
MediaTek        3
Qualcomm        1
NIVIDIA         1
ARM             1
Microsoft       1
ATI             1
Name: count, dtype: int64

Theo kết quả tìm kiếm trên 2 trang web https://pc-builds.com/vi/gpu/0MN/amd-radeon-r5-m230 và https://www.thegioididong.com/hoi-dap/danh-gia-suc-manh-cua-card-amd-radeon-pro-555x-1225183 thì 2 dòng GPU R5 và Pro 555X thuộc về hãng AMD, nên sẽ điền các giá trị khuyết này là 'AMD'


In [109]:
# Điền các giá trị khuyết trong GPU_Brand bằng 'AMD'
df['GPU_Brand'] = df['GPU_Brand'].fillna('AMD')

In [110]:
# Thống kê số lượng giá trị khuyết còn lại trong toàn bộ dữ liệu
df.isna().sum()

Unnamed: 0         0
Brand              0
Name               0
Price              0
Processor_Name     0
Processor_Brand    0
RAM_Expandable     0
RAM                0
RAM_TYPE           0
Ghz                0
Display_type       0
Display            0
GPU                0
GPU_Brand          0
SSD                0
HDD                0
Adapter            0
dtype: int64

#### Xử lý định dạng của giá trị của các trường cần thiết


Do dữ liệu được crawl về và chưa được xử lý nên vẫn còn ở dạng chuỗi, trong khi một số trường nên ở kiểu số như RAM_Expandable, RAM, Ghz... Phần này sẽ xử lý các trường về đúng định dạng của nó.


In [111]:
# Kiểm tra giá trị hiện có của cột Brand
df['Brand'].unique()

array(['HP', 'Lenovo', 'Dell', 'Infinix', 'Acer', 'ASUS', 'Tecno',
       'Xiaomi', 'MSI', 'Samsung', 'Apple', 'Realme', 'AVITA',
       'Zebronics', 'Honor', 'Fujitsu', 'Wings', 'LG', 'Gigabyte',
       'Microsoft', 'Razer', 'Coconics', 'Huawei', 'Micromax', 'iball',
       'Lava', 'VAIO', 'Smartron', 'Colorful', 'Nokia', 'RDP'],
      dtype=object)

Do Brand có ý nghĩa là thương hiệu của laptop, nên giá trị của nó sẽ là các tên thương hiệu và ở dạng chuỗi


In [112]:
# Kiểm tra giá trị hiện có của cột Name
df['Name'].unique()

array(['HP Chromebook 11A-NA0002MU (2E4N0PA) Laptop (11.6 Inch | MediaTek Octa Core | 4 GB | Google Chrome |)::585119::computer::laptops',
       'Lenovo Ideapad Slim 3 (82KU017KIN) Laptop (15.6 Inch | AMD Hexa Core Ryzen 5 | 8 GB | Windows 11 | 512 GB SSD)::594497::computer::laptops',
       'Dell G15-5520 (D560822WIN9B) Laptop (15.6 Inch | Core i5 12th Gen | 16 GB | Windows 11 | 512 GB SSD)::595299::computer::laptops',
       ...,
       'ASUS VivoBook Pro 15 OLED M3500QC-L1711WS Laptop (15.6 Inch | AMD Octa Core Ryzen 7 | 16 GB | Windows 11 | 1 TB SSD)::615984::computer::laptops',
       'ASUS TUF FX505DT-AL118T Laptop (15.6 Inch | AMD Quad core Ryzen 5 | 8 GB | Windows 10 | 512 GB SSD)::575591::computer::laptops',
       'Lenovo Legion 5i (82AU00CFIN) Laptop (15.6 Inch | Core i5 10th Gen | 8 GB | Windows 10 | 512 GB SSD)::581543::computer::laptops'],
      shape=(3941,), dtype=object)

Ngoài việc loại bỏ các chuỗi dư thừa trong các giá trị trên thì vẫn có thể tách một thông tin khác thành một cột riêng biệt là hệ điều hành. Như vậy, các công việc sẽ được thực hiện để định dạng giá trị cột Name như sau:

- Loại bỏ các chuỗi dư thừa, như là '::585119::computer::laptops'
- Tách chuỗi hệ điều hành sang cột mới là OS.
- Loại bỏ Model Number và Part Number, chẳng hạn như 82KU017KIN hay 11A-NA0002MU, các chuỗi này thường có ý nghĩa cụ thể về cấu hình hoặc phiên bản sản phẩm. Vì ý nghĩa của chuỗi này phụ thuộc vào cách mã hóa mà nhà sản xuất áp dụng, nên sẽ không có nhiều ý nghĩa hoặc ảnh hưởng tới giá bán của laptop.


In [113]:
# Hàm loại bỏ chuỗi dư thừa ::585119::computer::laptops
def remove_extra_text(df: pd.DataFrame, column: str):
    """Loại bỏ chuỗi dư thừa sau dấu '::' trong cột Name."""
    # sử dụng regex để loại bỏ chuỗi bắt đầu bằng "::" đết hết
    df[column] = df[column].str.replace(r'::.*$', '', regex=True)
    return df

In [114]:
def extract_os(df: pd.DataFrame, column: str):
    """
    Tách thông tin cấu hình từ cột Name, lấy hệ điều hành (thông tin thứ 4) để tạo cột OS.

    Parameters:
        df (pd.DataFrame): Dữ liệu chứa cột Name.
        column (str): Tên cột chứa dữ liệu cần xử lý.

    Returns:
        pd.DataFrame: DataFrame với cột mới 'OS'.
    """
    
    # Tách tất cả các phần nằm trong ngoặc ()
    df['Temp'] = df[column].str.findall(r'\((.*?)\)')
    
    # Lấy cặp ngoặc cuối cùng nếu có nhiều cặp
    df['Temp'] = df['Temp'].apply(lambda x: x[-1] if len(x) > 1 else x[0] if x else None)
    
    # loại bỏ chuỗi trong cặp () cuối cùng ra khỏi df[column]
    df[column] = df[column].str.replace(r'\((.*?)\)', '', regex=True)

    # Tách thông tin cấu hình theo dấu '|' và lấy hệ điều hành (thứ 4 trong danh sách)
    df['OS'] = df['Temp'].str.split('|').apply(lambda x: x[3].strip() if len(x) > 3 else None)

    # Loại bỏ cột Temp
    df.drop(columns=['Temp'], inplace=True)
    
    return df

In [115]:
def extract_model_name(df: pd.DataFrame, column: str):
    """
    Tách tên dòng máy từ cột dữ liệu bằng cách loại bỏ từ 'Laptop' và chuỗi có dấu '-'.

    Parameters:
        df (pd.DataFrame): Dữ liệu chứa cột cần xử lý.
        column (str): Tên cột chứa thông tin cần trích xuất.

    Returns:
        pd.DataFrame: DataFrame với cột 'Model Name' chứa tên dòng máy.
    """
    # Loại bỏ từ 'Laptop'
    df[column] = df[column].str.replace(r'\bLaptop\b', '', regex=True).str.strip()

    # Loại bỏ chuỗi có dấu '-'
    # df['Name'] = df['Temp'].str.replace(r'\b\w+-\w+\b', '', regex=True).str.strip()

    # Loại bỏ cột tạm thời
    # df.drop(columns=['Temp'], inplace=True)
    
    return df

In [116]:
# hàm trích xuất hệ điều hành và mẫu máy
def get_os_and_model(df: pd.DataFrame, column: str):
    """
    Tách hệ điều hành và tên dòng máy từ cột Name.
    
    Parameters:
        df (pd.DataFrame): Dữ liệu chứa cột Name.
        name_col (str): Tên cột chứa dữ liệu như 'Name'.

    Returns:
        pd.DataFrame: DataFrame với hai cột mới 'OS' và 'Model'.
    """
    df = remove_extra_text(df, column)
    df = extract_os(df, column) 
    df = extract_model_name(df, column)
    
    return df  
    

In [117]:
df = get_os_and_model(df, 'Name')
df

,Unnamed: 0,Brand,Name,Price,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,OS
0,0,HP,HP Chromebook 11A-NA0002MU,22990,MediaTek Octa-core,MediaTek,Not Expandable,4 GB,DDR4 RAM,2.0 Ghz Processor,LED,11.6,Integrated Graphics,MediaTek,64 GB SSD Storage,No HDD,45,Google Chrome
1,1,Lenovo,Lenovo Ideapad Slim 3,36289,AMD Hexa-Core Ryzen 5,AMD,12 GB Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LCD,15.6,Radeon,AMD,512 GB SSD Storage,No HDD,65,Windows 11
2,3,Dell,Dell G15-5520,78500,Intel Core i5 (12th Gen),Intel,32 GB Expandable,16 GB,DDR5 RAM,3.3 Ghz Processor,LCD,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,56,Windows 11
3,4,HP,HP 15s-fy5007TU,55490,Intel Core i5 (12th Gen),Intel,8 GB Expandable,8 GB,DDR4 RAM,4.2 Ghz Processor,LCD,15.6,Iris Xe,Intel,512 GB SSD Storage,No HDD,no,Windows 11
4,6,Infinix,Infinix Inbook Y2 Plus XL29,21990,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB LP,LPDDR4X RAM,1.7 Ghz Processor,LCD,15.6,UHD,Intel,512 GB SSD Storage,No HDD,45,Windows 11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3971,4403,ASUS,ASUS Zenbook UX430UN-GV022T,84990,Intel Core i5 (8th Gen),Intel,32 GB Expandable,8 GB,DDR4 RAM,1.6 Ghz Processor,LED,14,"Geforce MX150 GPU, 2 GB",NVIDIA,512 GB SSD Storage,No HDD,65,Windows 10
3972,4404,ASUS,ASUS Vivobook K15 OLED K513EA-L301WS,40031,Intel Core i3 (11th Gen),Intel,Not Expandable,8 GB,DDR4 RAM,4.0 Ghz Processor,LED,15.6,UHD,Intel,256 GB SSD Storage,No HDD,65,Windows 11
3973,4405,ASUS,ASUS VivoBook Pro 15 OLED M3500QC-L1711WS,75990,AMD Octa-Core Ryzen 7,AMD,Not Expandable,16 GB,DDR4 RAM,3.1 Ghz Processor,LED,15.6,"GeForce RTX 3050 GPU, 4 GB",NVIDIA,1024 GB SSD Storage,No HDD,120,Windows 11
3974,4407,ASUS,ASUS TUF FX505DT-AL118T,71909,AMD Quad-Core Ryzen 5,AMD,32 GB Expandable,8 GB,DDR4 RAM,2.1 Ghz Processor,LED,15.6,"GeForce GTX 1650 GPU, 4 GB",NVIDIA,512 GB SSD Storage,No HDD,150,Windows 10


Sau khi xử lý cột Name ta có được cột OS chứa hệ điều hành và cột Name chứa Tên dòng máy


Ta có cột Price_VND thay cho Price


In [118]:
# chuyển đổi cột Price từ đơn vị Rupee sang VND
df['Price_VND'] = df['Price'] * 300 # 1 rupee gần bằng 300 vnd
# Loại bỏ cột Price
df = df.drop('Price', axis=1)

In [119]:
# Xem xét các giá trị hiện có của trường Processor_Name
df['Processor_Name'].value_counts()

Processor_Name
Intel Core i5 (11th Gen)           316
Intel Core i5 (12th Gen)           294
AMD Hexa-Core Ryzen 5              245
AMD Octa-Core Ryzen 7              239
Intel Core i3 (11th Gen)           189
                                  ... 
Microsoft SQ1 Processor              1
AMD Dual-Core A9 APU Processor       1
AMD Octa-Core Ryzen 9 Processor      1
1.9 Ghz Processor                    1
2.6 Ghz Processor                    1
Name: count, Length: 125, dtype: int64

In [120]:
# đếm số lượng các dòng có chứa 'Ghz' trong cột Processor_Name
df['Processor_Name'].str.contains('Ghz').value_counts()

Processor_Name
False    3955
True       21
Name: count, dtype: int64

Tương tự như cách xử lý của cột Battery_Life, ở cột Processor_Name này, có một số dòng có giá trị như là 1.6 Ghz Processor...Những giá trị này không thể hiện được sự khác biệt giữa các dòng vi xử lý, nên sẽ xem nó như giá trị null và loại bỏ.


In [121]:
# Chuyển các giá trị chứa 'Ghz Processor' thành NaN trong cột Processor_Name
df['Processor_Name'] = df['Processor_Name'].apply(lambda x: np.nan if 'Ghz Processor' in str(x) else x)

# lọc bỏ các dòng đã chuyển thành null
df = df.dropna(subset=['Processor_Name'])

# kiểm tra giá trị khuyết còn lại trong cột
df['Processor_Name'].isna().sum()


np.int64(0)

In [122]:
# kiểm tra giá trị của cột Processor_Brand
df['Processor_Brand'].unique()

array(['MediaTek', 'AMD', 'Intel', 'Apple', 'Qualcomm', 'Microsoft'],
      dtype=object)

In [123]:
# Kiểm tra giá trị của cột RAM_Expandable
df['RAM_Expandable'].unique()

array(['Not Expandable', ' 12 GB Expandable', ' 32 GB Expandable',
       ' 8 GB Expandable', ' 16 GB Expandable', ' 4 GB Expandable',
       ' 64 GB Expandable', ' 48 GB Expandable', ' 2 GB Expandable',
       ' 10 GB Expandable'], dtype=object)

Giá trị của cột này nên là kiểu int. Giá trị Not Expandable sẽ được dán là 0


In [124]:
# Định dạng giá trị của RAM_Expandable bằng cách loại bỏ chuỗi 'GB Expandable'
df['RAM_Expandable'] = df['RAM_Expandable'].str.replace(r'GB Expandable|Expandable', '', regex=True).str.strip()

# Chuyển các giá trị 'Not' thành 0 
df['RAM_Expandable'] = df['RAM_Expandable'].replace('Not', '0')

# chuyển kiếu dữ liệu từ chuỗi thành số
df['RAM_Expandable'] = df['RAM_Expandable'].astype(int)
df.dtypes


Unnamed: 0          int64
Brand              object
Name               object
Processor_Name     object
Processor_Brand    object
RAM_Expandable      int64
RAM                object
RAM_TYPE           object
Ghz                object
Display_type       object
Display            object
GPU                object
GPU_Brand          object
SSD                object
HDD                object
Adapter            object
OS                 object
Price_VND           int64
dtype: object

In [125]:
# Kiểm tra giá trị của cột RAM
df['RAM'].unique()

array(['4 GB ', '8 GB ', '16 GB ', '8 GB LP', '16 GB LP', '4 GB LP',
       '32 GB LP', '8 GB RAM', '32 GB ', '16 GB RAM', '16 GB RAM ',
       '18 GB RAM', '32 GB RAM', '64 GB ', '12 GB ', '36 GB RAM', '2 GB ',
       '24 GB ', '6 GB ', '3 GB '], dtype=object)

Xử lý cột RAM tương tự như cột RAM_Expandable


In [126]:
# loại bỏ đi các ký tự không phải số
df['RAM'] = df['RAM'].str.replace(r'GB RAM|GB LP|GB|', '', regex=True).str.strip()

# Chuyển kiểu dữ liệu thành int
df['RAM'] = df['RAM'].astype(int)
df.dtypes

Unnamed: 0          int64
Brand              object
Name               object
Processor_Name     object
Processor_Brand    object
RAM_Expandable      int64
RAM                 int64
RAM_TYPE           object
Ghz                object
Display_type       object
Display            object
GPU                object
GPU_Brand          object
SSD                object
HDD                object
Adapter            object
OS                 object
Price_VND           int64
dtype: object

In [127]:
# kiểm tra giá trị của cột RAM_TYPE
df['RAM_TYPE'].unique()

array([' DDR4 RAM', ' DDR4 RAM ', ' DDR5 RAM ', ' LPDDR4X RAM',
       ' LPDDR5X RAM ', ' LPDDR5 RAM', ' LPDDR4 RAM', ' DDR5 RAM',
       ' LPDDR5X RAM', ' LPDDR5 RAM ', ' LPDDR4X RAM ', ' LPDDR3 RAM',
       ' RAM', ' LPDDR4 RAM ', ' DDR3 RAM', ' RAM ', ' DDR3 RAM ',
       ' LPDDR3 RAM ', ' DDR2 RAM'], dtype=object)

Ta thấy xuất hiện giá trị chỉ có RAM, không thể hiện được là loại RAM nào, vì vậy phải xem số lượng dòng chỉ có mỗi 'RAM' để quyết định xem là loại bỏ hay là điền khuyết


In [128]:
# Đếm số lượng giá trị "RAM" trong cột RAM_TYPE
ram_count = df[df['RAM_TYPE'].str.strip() == 'RAM'].shape[0]

# Đếm tổng số dòng trong cột RAM_TYPE
total_count = df['RAM_TYPE'].shape[0]

# Tính tỷ lệ "RAM"
ram_ratio = ram_count / total_count * 100

# In ra tỷ lệ
print(f"Tỷ lệ 'RAM' trong cột RAM_TYPE: {ram_ratio:.2f}%")


Tỷ lệ 'RAM' trong cột RAM_TYPE: 0.99%


Với kết quả trên, ta có thể xem các giá trị chỉ có 'RAM' là giá trị null và lọc bỏ


In [129]:
# Chuyển các giá trị chỉ chứa 'RAM' thành NaN trong cột RAM_TYPE
df['RAM_TYPE'] = df['RAM_TYPE'].apply(lambda x: np.nan if x.strip()=='RAM' else x)

# lọc bỏ giá trị null
df = df.dropna(subset=['RAM_TYPE'])

# kiểm tra giá trị khuyết còn lại trong cột
df['RAM_TYPE'].isna().sum()

np.int64(0)

In [130]:
# kiểm tra giá trị của cột Ghz
df['Ghz'].unique()

array([' 2.0 Ghz Processor', ' 4.0 Ghz Processor', ' 3.3 Ghz Processor',
       ' 4.2 Ghz Processor', ' 1.7 Ghz Processor', '0',
       ' 2.5 Ghz Processor', ' 2.2 Ghz Processor', ' 2.9 Ghz Processor',
       ' 3.4 Ghz Processor', ' 2.6 Ghz Processor', ' 3.0 Ghz Processor',
       ' 4.7 Ghz Processor', ' 3.1 Ghz Processor', ' 2.7 Ghz Processor',
       ' 2.8 Ghz Processor', ' 2.4 Ghz Processor', ' 2.1 Ghz Processor',
       ' 4.8 Ghz Processor', ' 1.2 Ghz Processor', ' 1.0 Ghz Processor',
       ' 1.8 Ghz Processor', ' 1.1 Ghz Processor', ' 2.3 Ghz Processor',
       ' 1.6 Ghz Processor', ' 0.8 Ghz Processor', ' 1.3 Ghz Processor',
       ' 1.5 Ghz Processor', ' 1.9 Ghz Processor', ' 5.5 Ghz Processor',
       ' 1.4 Ghz Processor'], dtype=object)

Cột Ghz trông có vẻ là biến phân loại vì có số lượng giá trị riêng biệt nhất định, nhưng các giá trị lại rời rạc và không có quy tắc nào, nên có thể xem như đây là một biến số chứa các giá trị liên tục. Ta sẽ xử lý cột này bằng cách xóa bỏ chuỗi 'Ghz Processor' và chuyển các giá trị số thành float.


In [131]:
# loại bỏ đi chuỗi 'Ghz Processor'
df['Ghz'] = df['Ghz'].str.replace('Ghz Processor', '', regex=False).str.strip()

# Chuyển kiểu dữ liệu thành int
df['Ghz'] = df['Ghz'].astype(float)
df.dtypes

Unnamed: 0           int64
Brand               object
Name                object
Processor_Name      object
Processor_Brand     object
RAM_Expandable       int64
RAM                  int64
RAM_TYPE            object
Ghz                float64
Display_type        object
Display             object
GPU                 object
GPU_Brand           object
SSD                 object
HDD                 object
Adapter             object
OS                  object
Price_VND            int64
dtype: object

In [132]:
# Kiểm tra thống kê mô tả hiện tại của Ghz
df['Ghz'].describe()

count    3916.000000
mean        2.626456
std         1.255342
min         0.000000
25%         1.800000
50%         2.400000
75%         3.400000
max         5.500000
Name: Ghz, dtype: float64

Ta thấy giá trị min là 0.0. Tuy nhiên, xung nhịp của vi xử lý không thể nào là 0. nên ta sẽ xem đây là giá trị không hợp lệ và xem nó là giá trị khuyết. Sau đó ta sẽ tiến hành điền lại giá trị này. Ta thấy rằng giá trị trung bình là là 2.6 và trung vị là 2.4, vì vậy phân phối của biến này có xu hướng lệch phải nhẹ, nhưng không quá lớn, vì vậy có thể điền bằng 1 trong 2 giá trị là mean và median.


In [133]:
# chuyển giá trị 0 thành nan
df['Ghz'] = df['Ghz'].replace(0.0, np.nan)

In [134]:
# Kiểm tra lại thống kê mô tả của Ghz
df['Ghz'].describe()

count    3724.000000
mean        2.761869
std         1.132722
min         0.800000
25%         2.000000
50%         2.500000
75%         3.400000
max         5.500000
Name: Ghz, dtype: float64

In [135]:
print(f"Tỉ lệ khuyết hiện tại của biến Ghz: {(df['Ghz'].isna().sum()/df.shape[0])*100.0}")

Tỉ lệ khuyết hiện tại của biến Ghz: 4.902962206332993


Ta thấy tỉ lệ khuyết hiện tại không quá đáng kể, nên dù điền bằng giá trị nào thì cũng sẽ không ảnh hưởng nhiều tới hiệu quả dự đoán. Hơn nữa, xung nhịp có thể thay đổi với nhiều giá trị trong quá trình sử dụng, nhưng chỉ nằm trong khoảng thông dụng như là khoảng 2.5 - 3.6 như laptop được sản xuất hiện nay, nên việc sử dụng giá trị median để điền khuyết sẽ là hợp lý trong trường hợp này.


In [136]:
# điền khuyết bằng median của Ghz
df['Ghz'] = df['Ghz'].fillna(df['Ghz'].median())

# Kiểm tra giá trị khuyết còn lại của Ghz
print(df['Ghz'].isnull().sum())  # Output: 0

0


In [137]:
# Xem giá trị của cột Display_type
df['Display_type'].unique()

array(['LED', 'LCD'], dtype=object)

In [138]:
# Xem giá trị của cột Display
df['Display'].unique()

array(['11.6 ', '15.6 ', '14 ', '16 ', '17.3 ', '13.3 ', '16.1 ', '14.1 ',
       '15.3 ', '13.4 ', '15 ', '10.5 ', '13.5 ', '13 ', '12.4 ',
       'OLED Display With Touchscreen', '14.0 ', '17 ', '18 ', '14.5 ',
       '12 ', '10.1 ', '15. ', '14.9 ', '11 ', '12.3 ', '16.6 ', '12.2 ',
       '12.5 ', '15.4 ', '13.6 ', '12.0 ', '16.2 '], dtype=object)

Cột Display thể hiện kích thước màn hình hiển thị, ta thấy có một giá trị không hợp lệ ở đây là 'OLED Display With Touchscreen'. Ta sẽ thay thể trị này


In [139]:
df[df['Display']=='OLED Display With Touchscreen']

,Unnamed: 0,Brand,Name,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,OS,Price_VND
774,817,HP,HP ENVY 15 ep1087TX,Intel Core i9 (11th Gen),Intel,0,32,DDR4 RAM,4.8,LED,OLED Display With Touchscreen,"GeForce RTX 3060 GPU, 6 GB",NVIDIA,1024 GB SSD Storage,No HDD,83,Windows 11,56820000


Ta có duy nhất 1 mẫu chứa giá trị 'OLED Display With Touchscreen'. Theo trang web https://hpworldkbm.com/product/hp-envy-laptop-15-ep1087tx/ thì máy tính này có kích thước màn hình là 15.6 inch, vì vậy ta sẽ điền bằng giá trị này


In [140]:
# điền lại giá trị khuyết
df['Display'] = df['Display'].str.replace('OLED Display With Touchscreen', '15.6').str.strip()

# chuyển lại thành giá trị float
df['Display'] = df['Display'].astype(float)

In [141]:
# xem giá trị của cột GPU
df['GPU'].unique()

array(['Integrated Graphics', 'Radeon', 'GeForce RTX 3050 GPU, 4 GB',
       'Iris Xe', 'UHD', 'UHD 620', 'GeForce RTX 2050 GPU, 4 GB',
       'GeForce RTX 4050 GPU, 6 GB', 'Arc', 'GeForce GTX 1650 GPU, 4 GB',
       'UHD 605', 'GeForce RTX 3060 GPU, 6 GB',
       'GeForce GTX 1650 Max Q GPU, 4 GB',
       'GeForce RTX 3050 Ti GPU, 4 GB', 'Integrated',
       'Geforce GTX 1650 GPU, 4 GB', 'Radeon Vega 3', 'UHD Graphics',
       'UHD GPU, 128 MB', 'M1', 'HD 6000', 'UHD 600', 'Radeon Vega 6',
       'HD 620', 'Geforce MX150 GPU, 2 GB', 'Radeon Vega 8',
       'Geforce GTX 1650 Max-Q GPU, 4 GB',
       'GeForce GTX 1650 Ti GPU, 4 GB', 'Radeon RX Vega 10', 'HD 520',
       'Radeon RX 5500M GPU, 4 GB', 'GeForce GTX 3050 GPU, 4 GB',
       'Radeon 610M', 'Radeon RX6500M GPU, 4 GB',
       'GeForce GTX 1650 MAX Q GPU, 4 GB', 'GeForce RTX 2050', 'HD 500',
       'Radeon Graphics', 'GeForce RTX 3070 Ti GPU, 8 GB',
       'GeForce GTX 1050 GPU, 4 GB', 'Geforce RTX 3050 GPU, 4 GB', 'M2',
       '

Ta sẽ có thêm 1 cột mới được tách ra từ cột GPU là VRAM. VRAM sẽ chứa các giá trị RAM ảo mà GPU đó hỗ trợ. loại GPU tích hợp sẽ không có VRAM, sẽ được gán giá trị là 0.


In [142]:
# Tách thông tin VRAM (GB) bằng regex
df['VRAM'] = df['GPU'].str.extract(r'(\d+\s?GB)')

# Chuẩn hóa giá trị VRAM để loại bỏ khoảng trắng
df['VRAM'] = df['VRAM'].str.strip()

# thay giá trị nan là 0 GB
df['VRAM'] = df['VRAM'].fillna('0 GB')
df['VRAM']

0       0 GB
1       0 GB
2       4 GB
3       0 GB
4       0 GB
        ... 
3971    2 GB
3972    0 GB
3973    4 GB
3974    4 GB
3975    4 GB
Name: VRAM, Length: 3916, dtype: object

In [143]:
df['VRAM'].unique()

array(['0 GB', '4 GB', '6 GB', '2 GB', '8 GB', '12 GB', '16 GB', '3 GB',
       '1 GB'], dtype=object)

In [144]:
# Loại bỏ chuỗi 'GB'
df['VRAM'] = df['VRAM'].str.replace(r'GB', '', regex=True).str.strip()

# Chuyển sang dạng số
df['VRAM'] = df['VRAM'].astype(int)

In [145]:
# Tách tên GPU
import re

# Điền giá trị mặc định nếu cột có NaN
df['GPU'] = df['GPU'].fillna('Unknown')

# Tách và thay đổi trực tiếp trên cột GPU
df['GPU'] = df['GPU'].str.extract(r'^(.*?\bGPU\b)', flags=re.IGNORECASE).fillna(df['GPU'])


In [146]:
# Xử lý định dạng tên GPU
# df['GPU'] = df['GPU'].str.replace('Integrated GPU', 'Integrated Graphics')
# df['GPU'] = df['GPU'].str.replace(r'GPU', '', regex=True).str.strip()
# df['GPU'] = df['GPU'].str.replace('Geforce GTX 1050Ti', 'GeForce GTX 1050Ti').str.replace('GeForce GTX 1050Ti', 'GeForce GTX 1050 Ti')
# df['GPU'] = df['GPU'].str.replace('geforce gtx 1650 max q', 'geforce gtx 1650 max-q')
# df['GPU'] = df['GPU'].str.replace('geforce gtx 1650ti', 'geforce gtx 1650 ti')
# df['GPU'] = df['GPU'].str.replace('radeon rx6500m', 'radeon rx 6500m')
# df['GPU'] = df['GPU'].str.replace('geforce rtx 3050ti', 'geforce rtx 3050 ti')
# df['GPU'] = df['GPU'].str.replace('geoforce rtx3050', 'geforce rtx 3050')
# df['GPU'] = df['GPU'].str.replace(' geforce mx350', 'mx350')
# df['GPU'] = df['GPU'].str.replace('geforce geforce geforce geforce geforce geforce geforce geforce rtx a500', 'rtx a500')




In [147]:
df['GPU'].unique()

array(['Integrated Graphics', 'GeForce RTX 3050 GPU',
       'GeForce RTX 2050 GPU', 'GeForce RTX 4050 GPU',
       'GeForce GTX 1650 GPU', 'GeForce RTX 3060 GPU',
       'GeForce GTX 1650 Max Q GPU', 'GeForce RTX 3050 Ti GPU',
       'Geforce GTX 1650 GPU', 'UHD GPU', 'Geforce MX150 GPU',
       'Geforce GTX 1650 Max-Q GPU', 'GeForce GTX 1650 Ti GPU',
       'Radeon RX 5500M GPU', 'GeForce GTX 3050 GPU',
       'Radeon RX6500M GPU', 'GeForce GTX 1650 MAX Q GPU',
       'GeForce RTX 3070 Ti GPU', 'GeForce GTX 1050 GPU',
       'Geforce RTX 3050 GPU', 'Radeon RX 6500M GPU',
       'Radeon RX 5600M GPU', 'GeForce MX450 GPU', 'GeForce RTX 4060 GPU',
       'GeForce GTX 1060 GPU', 'GeForce MX330 GPU',
       'GeForce RTX 3050 Ti  GPU', 'GeForce MX350 GPU',
       'Radeon RX 6800M GPU', 'GeForce MX250 GPU', 'GeForce RTX 3070 GPU',
       'MX350 GPU', 'GeForce MX550 GPU', 'Geforce GTX 1650 Ti GPU',
       'GEFORCE RTX 3050 Ti GPU', 'Radeon RX 6700S GPU',
       'GeForce RTX 3080 GPU', 'GeFor

Vì giá trị của cột này có quà nhiều giá trị khác biệt, đồng thời cũng chứa rất nhiều chuỗi bị sai định dạng, nên nhóm sẽ xuất ra file txt để tiến hành chỉnh sửa thủ công.


In [148]:
# xuất file txt
# gpu = df['GPU'].to_csv("gpu.txt", index=False, header=False)

In [149]:
# đọc lại file gpu đã xử lý 
gpu_data = pd.read_csv("gpu.txt", header=None, names=['GPU'])
# reset index để tránh bị lệch
df = df.reset_index(drop=True)
df['GPU'] = gpu_data['GPU']

In [150]:
# Loại bỏ chuỗi 'GPU'
df['GPU'] = df['GPU'].str.replace(r'GPU', '', regex=True).str.strip()

In [151]:
df.isna().sum()

Unnamed: 0         0
Brand              0
Name               0
Processor_Name     0
Processor_Brand    0
RAM_Expandable     0
RAM                0
RAM_TYPE           0
Ghz                0
Display_type       0
Display            0
GPU                0
GPU_Brand          0
SSD                0
HDD                0
Adapter            0
OS                 0
Price_VND          0
VRAM               0
dtype: int64

In [152]:
df['GPU'].unique()

array(['Integrated Graphics', 'GeForce RTX 3050', 'GeForce RTX 2050',
       'GeForce RTX 4050', 'GeForce GTX 1650', 'GeForce RTX 3060',
       'GeForce GTX 1650 Max Q', 'GeForce RTX 3050 Ti', 'UHD',
       'GeForce MX150', 'GeForce GTX 1650 Ti', 'Radeon RX 5500M',
       'GeForce GTX 3050', 'Radeon RX6500M', 'GeForce GTX 1650 MAX Q',
       'GeForce RTX 3070 Ti', 'GeForce GTX 1050', 'Radeon RX 6500M',
       'Radeon RX 5600M', 'GeForce MX450', 'GeForce RTX 4060',
       'GeForce GTX 1060', 'GeForce MX330', 'GeForce MX350',
       'Radeon RX 6800M', 'GeForce MX250', 'GeForce RTX 3070',
       'GeForce MX550', 'Radeon RX 6700S', 'GeForce RTX 3080',
       'GeForce MX570', 'GeForce 940MX', 'GeForce RTX 4080',
       'GeForce RTX 4070', 'GeForce RTX 2070', 'GeForce N16V-GMR1',
       'Arc A370M', 'Radeon RX6550M', 'GeForce RTX 3080 Ti', 'Radeon',
       'Radeon 680M', 'GeForce RTX 4090', 'RTX A500', 'Quadro T1200',
       'GeForce RTX 2080', 'Quadro T2000', 'Quadro T50', 'GeForce P620',
 

In [153]:
# Xem dữ liệu cốt GPU_Brand
df['GPU_Brand'].unique()

array(['MediaTek', 'AMD', 'NVIDIA', 'Intel', 'Apple', 'Qualcomm',
       'NIVIDIA', 'Nvidia', 'ARM', 'Microsoft', 'ATI'], dtype=object)

In [154]:
df['GPU_Brand'] = df['GPU_Brand'].replace(['Nvidia', 'NIVIDIA'], 'NVIDIA')

In [155]:
# xem giá trị của SSD
df['SSD'].value_counts()

SSD
512 GB SSD Storage     1907
NO SSD                  652
1024 GB SSD Storage     576
256 GB SSD Storage      349
256 GB SSD Storage      205
128 GB SSD Storage       57
128 GB SSD Storage       47
2048 GB SSD Storage      34
32 GB SSD Storage        25
64 GB SSD Storage        23
512 GB SSD Storage       16
16 GB SSD Storage        12
8 GB SSD Storage          4
512  SSD Storage          2
16 GB SSD Storage         2
256  SSD Storage          2
1 GB SSD Storage          1
4098 GB SSD Storage       1
32 GB SSD Storage         1
Name: count, dtype: int64

In [156]:
# Loại bỏ chuỗi 'GB SSD Storage'
df['SSD'] = df['SSD'].str.replace(r'GB SSD Storage', '', regex=True).str.strip()

# Loại bỏ chuỗi 'SSD Storage còn lại'
df['SSD'] = df['SSD'].str.replace(r' SSD Storage', '', regex=True).str.strip()

# Chuyển 'NO SSD' thành 0
df['SSD'] = df['SSD'].replace('NO SSD', '0').str.strip()

# chuyển về kiểu int
df['SSD'] = df['SSD'].astype(int)

df['SSD'].value_counts()

SSD
512     1925
0        652
1024     576
256      556
128      104
2048      34
32        26
64        23
16        14
8          4
1          1
4098       1
Name: count, dtype: int64

In [157]:
# Lọc ra các cột có SSD < 128 
df_filtered = df[(df['SSD'] < 64) & (df['SSD'] > 0)]
df_filtered['Name'].unique()

array(['ASUS ROG Flow X13 GV301QH-K5460TS', 'HP Pavilion x360 14-ba073TX',
       'Lenovo Ideapad 330-15ICH',
       'HP Pavilion TouchSmart 14 X360 14-cd0051TU',
       'Acer Nitro 5 AN515-52-76VR', 'Acer Aspire Switch One SW110-1CT',
       'Samsung Chromebook XE500C13-K04US',
       'Micromax Canvas Lapbook   L1161 Netbook',
       'Micromax Canvas Lapbook L1160', 'Acer Chromebook CB3-431-C5FM',
       'iball Excelance CompBook', 'iball CompBook M500',
       'Acer Nitro 5 AN515-52-593F', 'Samsung Chromebook XE500C13-K05US',
       'ASUS Vivobook L402WA-EH21', 'iball CompBook Merit G9',
       'ASUS Chromebook C223NA-GJ0074', 'Lava Helium 14',
       'Acer Swift 3 SF315-52G-52XD', 'Acer Swift 3 SF315-52G',
       'ASUS Vivobook E402NA-GA022T',
       'HP Pavilion TouchSmart 14 x360 14-cd0053TX',
       'ASUS EeeBook E203MA-FD014T', 'Micromax Canvas Laptab II LT777W',
       'iball Compbook Excelance-OHD', 'ASUS Chromebook C423NA-DH02',
       'ASUS VivoBook E12 E203NA-FD087T', 'ibal

Ta thấy rằng ở cột SSD có một số giá trị khá nhỏ như là 1. Đó là điều bất hợp lý vì laptop không thể chỉ có 1GB lưu trữ. Khi xem xét tên các dòng máy có dung lượng lưu trữ thấp dưới 64 GB thì ta nhận thấy rằng chủ yếu là các dòng tablet lai laptop. nên bộ nhớ thấp là điều chấp nhận được. Vì vậy, ta sẽ điền lại các giá trị nhỏ bằng dung lượng lưu trữ tối thiểu nhất là 64 GB.


In [158]:
df['SSD'] = df['SSD'].apply(lambda x: x if x >=64 else 64)
df['SSD'].value_counts()

SSD
512     1925
64       720
1024     576
256      556
128      104
2048      34
4098       1
Name: count, dtype: int64

In [159]:
# xem giá trị của biến hdd
df['HDD'].value_counts()

HDD
No HDD                  2967
1024 GB HDD Storage      593
 1024 GB HDD Storage     293
500 GB HDD Storage        40
2048 GB HDD Storage       18
 500 GB HDD Storage        3
 2048 GB HDD Storage       1
1 GB HDD Storage           1
Name: count, dtype: int64

In [160]:
# xử lý giá trị của biến HDD
# loại bỏ chuỗi GB HDD Storage
df['HDD'] = df['HDD'].str.replace(r'GB HDD Storage', '', regex=True).str.strip()

# chuyển NO HDD thành 0
df['HDD'] = df['HDD'].str.replace('No HDD', '0').str.strip()

# chuyển giá trị thành số
df['HDD'] = df['HDD'].astype(int)
df['HDD'].value_counts()

HDD
0       2967
1024     886
500       43
2048      19
1          1
Name: count, dtype: int64

In [161]:
# xem dữ liệu có HDD là 1
df[df['HDD'] == 1]

,Unnamed: 0,Brand,Name,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,OS,Price_VND,VRAM
3599,4076,Lenovo,Lenovo ThinkBook 14,Intel Core i3 (10th Gen),Intel,0,4,DDR4 RAM,2.1,LED,14.0,Integrated Graphics,Intel,64,1,65,DOS,11699700,0


Ta thấy rằng dòng máy này đã có SSD là 64, vì vậy ta sẽ chuyển 1 HDD thành 0


In [162]:
df['HDD'] = df['HDD'].replace(1,0)
df['HDD'].value_counts()

HDD
0       2968
1024     886
500       43
2048      19
Name: count, dtype: int64

In [163]:
# Xem giá trị của cột Adapter
df['Adapter'].unique()

array(['45', '65', '56', 'no', '150', '170', '230', '135', '41', '100',
       '90', '120', '96', '180', '63', '30', '54', '33', '46', '57', '43',
       '48', '70', '37', '24', '67', '42', '330', '200', '58', '240',
       '50', '280', '60', '80', '35', '51', '61', '38', '86', '66', '210',
       '39', '97', '83', '59', '53', '71', '75', '74', '49', '300', '140',
       '76', '68', '52', '40', '36', '32', '15', '26', '47', '64', '130',
       '87', '72', '95', '69'], dtype=object)

In [164]:
# xem dữ liệu có Adapter là no
df[df['Adapter']=='no']

,Unnamed: 0,Brand,Name,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,OS,Price_VND,VRAM
3,4,HP,HP 15s-fy5007TU,Intel Core i5 (12th Gen),Intel,8,8,DDR4 RAM,4.2,LCD,15.6,Integrated Graphics,Intel,512,0,no,Windows 11,16647000,0
15,18,Dell,Dell Vostro 3420,Intel Core i5 (12th Gen),Intel,16,8,DDR4 RAM,3.3,LCD,14.0,Integrated Graphics,Intel,512,0,no,Windows 11,15840000,0
19,22,Lenovo,Lenovo Ideapad Gaming 3 15IHU6,Intel Core i5 (11th Gen),Intel,16,8,DDR4 RAM,3.1,LCD,15.6,GeForce RTX 2050,NVIDIA,512,0,no,Windows 11,14970000,4
20,23,HP,HP Pavilion 14-dv2153TU,Intel Core i5 (12th Gen),Intel,16,16,DDR4 RAM,4.2,LCD,14.0,Integrated Graphics,Intel,1024,0,no,Windows 11,20999700,0
24,27,Lenovo,Lenovo Ideapad Slim 3 15ITL6,Intel Core i5 (11th Gen),Intel,16,16,DDR4 RAM,2.5,LCD,15.6,Integrated Graphics,Intel,512,0,no,Windows 11,12297000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3885,4377,ASUS,ASUS VivoBook Ultra K513EA-BQ501TS,Intel Core i5 (11th Gen),Intel,0,8,DDR4 RAM,2.4,LED,15.6,Integrated Graphics,Intel,512,0,no,Windows 10,13799700,0
3889,4381,Microsoft,Microsoft Surface Book,Intel Core i5 (7th Gen),Intel,0,8,DDR3 RAM,2.6,LED,13.5,Integrated Graphics,Intel,256,0,no,Windows 10,34499700,0
3895,4387,Dell,Dell G7 15 7590,Intel Core i7 (9th Gen),Intel,32,16,DDR4 RAM,2.6,LED,15.6,GeForce RTX 2060,NVIDIA,512,0,no,Windows 10,41997000,6
3905,4397,ASUS,ASUS ROG Strix Scar 17 G732LXS-HG010T,Intel Core i7 (10th Gen),Intel,0,16,DDR4 RAM,2.3,LED,17.3,GeForce RTX 2080 Super,NVIDIA,1024,0,no,Windows 10,54297000,8


In [165]:
# chuyển các dòng có giá trị là no thành nan
df['Adapter'] = df['Adapter'].replace('no', np.nan)

In [166]:
# Chuyển cột 'Adapter' thành số, bỏ qua NaN
df['Adapter'] = pd.to_numeric(df['Adapter'], errors='coerce')

In [167]:
df['Adapter'].value_counts()

Adapter
65.0     1428
45.0      462
41.0      155
150.0      91
135.0      80
         ... 
97.0        1
15.0        1
47.0        1
26.0        1
69.0        1
Name: count, Length: 67, dtype: int64

In [168]:
df['Adapter'].describe()

count    3143.000000
mean       80.473115
std        55.477372
min        15.000000
25%        45.000000
50%        65.000000
75%        65.000000
max       330.000000
Name: Adapter, dtype: float64

Ta thấy rằng giá trị trung vị là 65W, và độ lệch chuẩn là 55W, tức là khoảng phân tán là từ 10W đến 120W. tuy nhiên, giá trị lớn nhất lại là 330W, có nghĩa là phân phối của biến này không đều, điền bằng giá trị trung vị sẽ thích hợp hơn. Thêm vào đó, 65W cũng là giá trị phổ biến nhất, vì vậy điền giá trị khuyết bằng 65 sẽ là thích hợp trong trường hợp này.


In [169]:
mode_value = df['Adapter'].mode()[0]
df['Adapter'] = df['Adapter'].fillna(mode_value)

In [170]:
# xem giá trị của cột OS
df['OS'].value_counts()

OS
Windows 11           2110
Windows 10           1508
DOS                   187
Google Chrome          31
Linux                  23
Windows 8.1            19
macOS Mojave           15
Ubuntu                  7
macOS Catalina          6
macOS High Sierra       3
macOS Sierra            3
macOS Big Sur           2
macOS Monterey          2
Name: count, dtype: int64

In [171]:
# xem giá trị của cột VRAM
df['VRAM'].value_counts()

VRAM
0     2455
4      684
6      281
2      269
8      181
16      24
12      14
3        6
1        2
Name: count, dtype: int64

In [172]:
# Chuyển Price_VND về thành cột cuối cùng vì đây là biến mục tiêu
column_to_move = df.pop('Price_VND')
df['Price_VND'] = column_to_move

In [173]:
# loại bỏ cột đầu tiên không có ý nghĩa
df.drop(columns=['Unnamed: 0'], inplace=True)

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            3916 non-null   object 
 1   Name             3916 non-null   object 
 2   Processor_Name   3916 non-null   object 
 3   Processor_Brand  3916 non-null   object 
 4   RAM_Expandable   3916 non-null   int64  
 5   RAM              3916 non-null   int64  
 6   RAM_TYPE         3916 non-null   object 
 7   Ghz              3916 non-null   float64
 8   Display_type     3916 non-null   object 
 9   Display          3916 non-null   float64
 10  GPU              3916 non-null   object 
 11  GPU_Brand        3916 non-null   object 
 12  SSD              3916 non-null   int64  
 13  HDD              3916 non-null   int64  
 14  Adapter          3916 non-null   float64
 15  OS               3916 non-null   object 
 16  VRAM             3916 non-null   int64  
 17  Price_VND     

Vì chỉ có duy nhất 1 giá trị có Brand là Razer nên chúng tôi loại bỏ giá trị này

In [175]:
df[df['Brand'] == 'Razer']

,Brand,Name,Processor_Name,Processor_Brand,RAM_Expandable,RAM,RAM_TYPE,Ghz,Display_type,Display,GPU,GPU_Brand,SSD,HDD,Adapter,OS,VRAM,Price_VND
559,Razer,Razer Blade 15 RZ09-0421NEC3-R3U1,Intel Core i7 (12th Gen),Intel,64,16,DDR5 RAM,1.8,LCD,15.6,GeForce RTX 3070 Ti,NVIDIA,1024,0,45.0,Windows 10,8,107699700


In [176]:
# Remove if brand is 'Razer'
df = df[~ (df['Brand'] == 'Razer')]

In [177]:
# Lưu data lại thành file csv mới
df.to_csv('../Data/processed_data.csv', index=False)